In [3]:
!pip install pandas
!pip install numpy
!pip install pyloudnorm
!pip install matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.2/162.2 kB 2.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.9/245.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 44.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 15.6 MB/s eta 0:00:00


## Data Wrangling for Mixes

In [7]:
%matplotlib inline
import numpy as np
import pandas as pd

df = pd.read_json("../final-track-data.json")
mixes_df = pd.json_normalize(df["mixes"])
mixes_df.head(5)

,mix-name,song-name,artist-name,genre,mix-evaluation,tracks
0,PXL-L1,Vermont,The Districts,POP ROCK,"[{'track-num-eval': 0.76, 'track-semantic-eval...","[{'track-name': 'Kick In', 'track-type': 'AUDI..."
1,PXL-L2,Vermont,The Districts,POP ROCK,"[{'track-num-eval': 0.79, 'track-semantic-eval...","[{'track-name': 'Kick IN', 'track-type': 'AUDI..."
2,PXL-L3,Vermont,The Districts,POP ROCK,"[{'track-num-eval': 0.51, 'track-semantic-eval...","[{'track-name': 'K in', 'track-type': 'AUDIO',..."
3,PXL-L4,Vermont,The Districts,POP ROCK,"[{'track-num-eval': 0.77, 'track-semantic-eval...","[{'track-name': 'KickIn_15', 'track-type': 'AU..."
4,PXL-L5,Vermont,The Districts,POP ROCK,"[{'track-num-eval': 0.66, 'track-semantic-eval...","[{'track-name': 'KickIn_15', 'track-type': 'AU..."


### Looping through the tracks for each mix

In [13]:
# Create a multiple dimensional array for the tracks in each mix
mixes = []
for i, row in mixes_df.iterrows():
    print(row)
    mixes.append(row["tracks"])
    
# print(mixes)

track_count = 0

for mix in mixes:
    for track in mix:
        track_count += 1
        
print(track_count)

mix-name                                                     PXL-L1
song-name                                                   Vermont
artist-name                                           The Districts
genre                                                      POP ROCK
mix-evaluation    [{'track-num-eval': 0.76, 'track-semantic-eval...
tracks            [{'track-name': 'Kick In', 'track-type': 'AUDI...
Name: 0, dtype: object
mix-name                                                     PXL-L2
song-name                                                   Vermont
artist-name                                           The Districts
genre                                                      POP ROCK
mix-evaluation    [{'track-num-eval': 0.79, 'track-semantic-eval...
tracks            [{'track-name': 'Kick IN', 'track-type': 'AUDI...
Name: 1, dtype: object
mix-name                                                     PXL-L3
song-name                                                   Vermont
ar

In [23]:
!pip install datasets
!pip install HfApi

In [70]:
import json
from datasets import Dataset, DatasetDict

# Load the JSON data
with open("../final-track-data.json", "r") as file:
    data = json.load(file)

# Extract the relevant information from the JSON
track_data = []
for mix in data["mixes"]:
    mix_name = mix["mix-name"]
    song_name = mix["song-name"]
    artist_name = mix["artist-name"]
    genre = mix["genre"]
    
    for track in mix["tracks"]:
        if track["track-type"] == "AUDIO":
            track_name = track["track-name"]
            track_type = track["track-type"]
            track_instrument_subtype = track["track-instrument-subtype"]
            track_instrument_type = track["track-instrument-type"]
            channel_mode = track["channel-mode"]
            track_audio_features = track["track-audio-features"]
            parameters = json.dumps(track["parameters"])
                
            track_data.append({
                "mix_name": mix_name,
                "song_name": song_name,
                "artist_name": artist_name,
                "genre": genre,
                "track_name": track_name,
                "track_type": track_type,
                "track_instrument_subtype": track_instrument_subtype,
                "track_instrument_type": track_instrument_type,
                "channel_mode": channel_mode,
                "track_audio_path": track_audio_features['track-audio-path'],
                "track_audio_sample_rate": track_audio_features['track-audio-sample-rate'],
                "track_audio_lufs": track_audio_features['track-audio-lufs'],
                "parameters": parameters
            })

In [72]:
# Create a Dataset from the extracted track data
dataset = Dataset.from_list(track_data)

shuffled_dataset = dataset.shuffle(seed=42)
shuffled_dataset["genre"][:10]

# Split the dataset into train, dev, and test sets
train_dataset = shuffled_dataset.select(range(1545))
dev_dataset = shuffled_dataset.select(range(1545, 1845))
test_dataset = shuffled_dataset.select(range(1845, 2345))

# Create a DatasetDict with the splits
dataset_dict = DatasetDict({
    "train": train_dataset,
    "dev": dev_dataset,
    "test": test_dataset
})

In [75]:
from huggingface_hub import HfApi

# Push the dataset to your Hugging Face account
api = HfApi()
username = "mclemcrew"  # Replace with your Hugging Face username
dataset_name = "mix-evaluation-dataset"  # Choose a name for your dataset

In [76]:
dataset_dict.push_to_hub(
    repo_id=f"{username}/{dataset_name}",
    private=False,  # Set to False if you want the dataset to be public
    token=""  # Replace with your Hugging Face token
)

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  2.46it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/mclemcrew/mix-evaluation-dataset/commit/df15a107cf86ed46793df859daf43d5f24b142fd', commit_message='Upload dataset', commit_description='', oid='df15a107cf86ed46793df859daf43d5f24b142fd', pr_url=None, pr_revision=None, pr_num=None)

In [6]:
# Number of each audio effect for each track
eq_count = 0
gate_count = 0
compression_count = 0
reverb_count = 0
delay_count = 0
flanger_count = 0
phaser_count = 0
chorus_count = 0
track_count = 0

for mix in mixes:
    for track in mix:
        track_count += 1
        for key, value in track.items():     
            if(key == "parameters"):
                if("eq" in value):
                    eq_count += 1
                if("gate" in value):
                    gate_count += len(value['gate'])
                if("compression" in value):
                    compression_count += len(value['compression'])
                if("reverb" in value):
                    reverb_count += len(value['reverb'])
                if("delay" in value):
                    delay_count += len(value['delay'])
                if("flanger" in value):
                    flanger_count += len(value['flanger'])
                if("phaser" in value):
                    phaser_count += len(value['phaser'])
                if("chorus" in value):
                    chorus_count += len(value['chorus'])

print(f"Number of EQs used: {eq_count}")
print(f"Number of gates used: {gate_count}")
print(f"Number of compressions used: {compression_count}")
print(f"Number of reverbs used: {reverb_count}")
print(f"Number of delays used: {delay_count}")
print(f"Number of flangers used: {flanger_count}")
print(f"Number of phasers used: {phaser_count}")
print(f"Number of choruses used: {chorus_count}")
print(f"Number of total tracks: {track_count}")

Number of EQs used: 1628
Number of gates used: 249
Number of compressions used: 1444
Number of reverbs used: 1465
Number of delays used: 124
Number of flangers used: 4
Number of phasers used: 6
Number of choruses used: 10
Number of total tracks: 2411


## Getting data in a dataframe for easy reading and analysis

#### Could one hot encode the vector
| EQ | Gate | Compression | Reverb | Delay | Flanger | Phaser | Chorus |
| --- | --- | --- | --- | --- | --- | --- | --- |
|   1 |   1 |    0 |    0 |    0 |    1 |    1 |    1 | 


#### Or for training with a neural network arch, could do something like this for the data:
| Track | Effects |
| --- | --- |
|   OVERHEADS |   [COMPRESSION, REVERB, DELAY] |



In [ ]:
# tracks_information = []

# for mix in mixes: 
#     for track in mix:
#         if "track-audio-path" in track: # We only want audio tracks (many ways to do this, but this is one)
#             track_info = {'track-instrument': track.get('track-instrument', 'Unknown'), 'track-instrument-type': track.get('track-instrument-type', 'Unknown')}
#             parameters = track["parameters"]
#             effects = ['eq', 'gate', 'compression', 'reverb', 'delay', 'flanger', 'phaser', 'chorus']
#             for effect in effects:
#                 track_info[effect] = 1 if effect in parameters else 0
#             tracks_information.append(track_info)

# df_tracks = pd.DataFrame(tracks_information)

# df_tracks.head(50)

In [ ]:
import soundfile as sf
import pyloudnorm as pyln

for mix in mixes: 
    for track in mix:
        if "track-audio-path" in track:
            print(track.get('track-name'))
            data, rate = sf.read("./." + track.get('track-audio-path')) # load audio (with shape (samples, channels))
            meter = pyln.Meter(rate) # create BS.1770 meter
            loudness = meter.integrated_loudness(data) # measure loudness
            print(loudness)

In [ ]:
# Make sure the sample rate is what we expect
import wave
import soundfile as sf

# for index, row in track_list_df.iterrows():
for mix in mixes: 
    for track in mix:
        if "track-audio-path" in track:
            audio_path = '.' + str(track['track-audio-path'])
            data, samplerate = sf.read(audio_path, dtype='float32')
            print(f"Sample rate: {samplerate}")
            print(f"Shape: {data}")
            track['track-audio-sample-rate'] = samplerate
            track['track-audio-data'] = data

In [ ]:
import json

# Load the original tracks.json file
with open('../tracks.json', 'r') as file:
    data = json.load(file)

# Iterate over each track and modify it
for mix in data['mixes']:
    for track in mix['tracks']:
        if "track-audio-path" in track:
            # Create the new 'track-audio-features' dictionary
            track_audio_features = {
                'track-audio-path': track.pop('track-audio-path', None)  # Remove the track-audio-path and get its value
            }
            # Add the 'track-audio-features' dictionary to the track
            track['track-audio-features'] = track_audio_features

# Write the updated data to a new file
with open('../updated_tracks.json', 'w') as file:
    json.dump(data, file, indent=2)

In [3]:
import json
import soundfile as sf
import pyloudnorm as pyln
import numpy as np
import os

# Load your JSON data
with open('../updated_tracks.json', 'r') as file:
    data = json.load(file)

for mix in data['mixes']:
    for track in mix['tracks']:
        if "track-audio-features" in track:
            # Get the full audio path
            previous_audio_path = str(track['track-audio-features']['track-audio-path'])
            audio_path = '../' + previous_audio_path[2:]

            print(audio_path)
            # Read the audio file to get sample rate and data
            track_data, samplerate = sf.read(audio_path, dtype='float32')

            meter = pyln.Meter(samplerate) # create BS.1770 meter
            loudness = meter.integrated_loudness(track_data) # measure loudness

            name_audio = previous_audio_path[8:len(previous_audio_path)-4] + '.npy'
            name_audio_dir = '../np-arrays/' + name_audio.split('/')[0]
            name_audio_file = '../np-arrays/' + name_audio

            # Create the directory if it doesn't exist
            os.makedirs(name_audio_dir, exist_ok=True)

            # Create the file for the np array
            open(name_audio_file, 'w').close()

            np.save(name_audio_file, track_data)

            print(samplerate, loudness) 
            # Create the new 'track-audio-features' dictionary
            track_audio_features = {
                'track-audio-path': previous_audio_path,
                'track-audio-sample-rate': samplerate,
                'track-audio-data': '../np-arrays/' + name_audio,
                'track-audio-lufs': loudness
            }

            # Add the 'track-audio-features' dictionary to the track
            track['track-audio-features'] = track_audio_features

# Write the updated data to a new file
with open('../updated_tracks_full.json', 'w') as file:
    json.dump(data, file, indent=2)

# print(data)

../audio/Vermont/01 K in.wav
88200 -21.164288599793807
../audio/Vermont/02 K out.wav
88200 -24.559226110731856
../audio/Vermont/03 S top.wav
88200 -24.43009887714256
../audio/Vermont/04 S bot.wav
88200 -26.439228681934114
../audio/Vermont/05 R tom.wav
88200 -32.554750610519484
../audio/Vermont/06 Fl tom.wav
88200 -30.82851936045886
../audio/Vermont/08 OH ride.wav
88200 -24.83936214743081
../audio/Vermont/09 OH c.wav
88200 -22.998441573792007
../audio/Vermont/07 OH hh.wav
88200 -23.66046006949744
../audio/Vermont/10 Rm hh.wav
88200 -18.970095372834596
../audio/Vermont/11 Rm ride.wav
88200 -17.693618089926552
../audio/Vermont/12 Tmbo.wav
88200 -28.91695340741545
../audio/Vermont/13 Tmbo 2x.wav
88200 -26.68405171270473
../audio/Vermont/14 B di.wav


KeyboardInterrupt: 

In [6]:
!pip install ffprobe

  Preparing metadata (setup.py) ... done
  Created wheel for ffprobe: filename=ffprobe-0.5-py3-none-any.whl size=3406 sha256=7e4d9e534c7b5ba096d418ffb1287a7c0d57f51e80e482fdd203223e3da38bbb
  Stored in directory: /Users/mclem/Library/Caches/pip/wheels/2c/cb/c1/10daee0c3fad04c9d900006cd0f24bdd47afb74a5c1c085795
Successfully built ffprobe


In [17]:
import json
import soundfile as sf
import pyloudnorm as pyln
import numpy as np
import os
from pydub import AudioSegment

sample_rate = 0

def detect_silence_lufs(audio_path, silence_threshold=-40.0, chunk_size=2):
    """
    Detects silence at the beginning of an audio file using LUFS.
    
    Parameters:
    - audio_path: Path to the audio file.
    - silence_threshold: Threshold in LUFS for silence detection.
    - chunk_size: Size of audio chunks to analyze.
    
    Returns:
    - start_time: Start time of the first non-silent chunk in seconds.
    """
    track_data, sample_rate = sf.read(audio_path, dtype='float32')
    silence_duration = 0
    start_time = 0
    counter = 0

    for i in range(0, len(track_data), chunk_size * sample_rate):
        chunk = track_data[i:i + chunk_size * sample_rate]
        meter = pyln.Meter(sample_rate) # create BS.1770 meter
        loudness = meter.integrated_loudness(chunk) # measure loudness
    
        if loudness < silence_threshold:
            silence_duration += chunk_size
        else:
            if silence_duration > 0:
                start_time = i / sample_rate
            break

    return start_time

def trim_audio(audio_path, start_time):
    """
    Trims the audio file to start from the specified start time.
    
    Parameters:
    - audio_path: Path to the audio file.
    - start_time: Start time in seconds.
    
    Returns:
    - trimmed_audio: Trimmed audio file.
    """
    audio = AudioSegment.from_wav(audio_path)
    start_time_ms = start_time * 1000  # Convert seconds to milliseconds
    trimmed_audio = audio[start_time_ms:]  # Trim from start_time_ms
    return trimmed_audio


# Load your JSON data
with open('../updated_tracks.json', 'r') as file:
    data = json.load(file)

for mix in data['mixes']:
    for track in mix['tracks']:
        if "track-audio-features" in track:
            # Get the full audio path
            previous_audio_path = str(track['track-audio-features']['track-audio-path'])

            name_audio = previous_audio_path[8:len(previous_audio_path)-4] + '.npy'
            name_audio_dir = '../audio/Truncated/' + name_audio.split('/')[0]
            name_audio_file = '../audio/Truncated/' + previous_audio_path[8:]
            
            print(previous_audio_path)

            # Create the directory if it doesn't exist
            os.makedirs(name_audio_dir, exist_ok=True)
             
            if os.path.exists(name_audio_file):
                continue  # Skip this file if it has been processed
            

            start_time = detect_silence_lufs('.' + previous_audio_path)
            print("Start time:", start_time)
            
            # print(previous_audio_path[len(previous_audio_path)-3:])

            trimmed_audio = trim_audio('.' + previous_audio_path, start_time)
            trimmed_audio.export(name_audio_file, format=previous_audio_path[len(previous_audio_path)-3:])


./audio/Vermont/01 K in.wav
./audio/Vermont/02 K out.wav
./audio/Vermont/03 S top.wav
./audio/Vermont/04 S bot.wav
./audio/Vermont/05 R tom.wav
./audio/Vermont/06 Fl tom.wav
./audio/Vermont/08 OH ride.wav
./audio/Vermont/09 OH c.wav
./audio/Vermont/07 OH hh.wav
./audio/Vermont/10 Rm hh.wav
./audio/Vermont/11 Rm ride.wav
./audio/Vermont/12 Tmbo.wav
./audio/Vermont/13 Tmbo 2x.wav
./audio/Vermont/14 B di.wav
./audio/Vermont/18 mGtr1.wav
./audio/Vermont/20 mGtr rm.wav
./audio/Vermont/21 rGtr1.wav
./audio/Vermont/22 rGtr2 rm.wav
./audio/Vermont/22 rGtr2 rm.wav
./audio/Vermont/23 rGtr3.wav
./audio/Vermont/23 rGtr3 rm.wav
./audio/Vermont/26 mVHmy1.wav
./audio/Vermont/27 rVHmy1.wav
./audio/Vermont/16 Wrlz hi.wav
./audio/Vermont/17 Wrlz lo.wav
./audio/Vermont/24 rV1.wav
./audio/Vermont/25 rV1 2x.wav
./audio/Vermont/24 rV1.wav
./audio/Vermont/25 rV1 2x.wav
./audio/Vermont/24 rV1.wav
./audio/Vermont/01 K in.wav
./audio/Vermont/03 S top.wav
./audio/Vermont/04 S bot.wav
./audio/Vermont/05 R tom.wav